# Analysis ML fitness for DNMT3A_R882H

### We need to calculate from the most frequent mutation the [N\tau, s, sigma]

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import math
import scipy.optimize
import scipy.integrate as it
from scipy import integrate

In [4]:
def mut_dict2mut_list(mut_dict):
    mut_list = []
    for k, v in mut_dict.items():
        mut_list.append(float(v[1]))
    return mut_list

In [5]:
def normalised_VAF_normalised_densities_list(trimmed_VAF_list, study_total, mu, binmethod):
    y1 = []
    for i in trimmed_VAF_list:
        VAFs = float(i)
        y = np.log(VAFs)
        y1.append(y)
        
    normed_value = study_total*2*mu
    hist, bins = np.histogram(y1, bins=binmethod, range=(min(y1),max(y1)))
    widths = np.diff(bins)
    bin_centres = (bins[:-1] + bins[1:])/2
    hist = np.array(hist, dtype=float)
    print(bins)
    print(hist)
    normalised_hist = hist/(normed_value*widths)
    log_hist_for_plot = np.log(normalised_hist)

    errors = error_bars(hist, normed_value, widths)
    
    VAF_densities=[]
    lower_error = []
    upper_error = []
    for i in errors[0]:
        lower_error.append(i)
    for i in errors[1]:
        upper_error.append(i)
    VAF_densities_zip = zip(bin_centres, log_hist_for_plot, lower_error, upper_error)
    for a, b, c, d in VAF_densities_zip:
        VAF_densities.append((a, b, c, d))
        
    return VAF_densities    

def error_bars(hist, normed_value, widths):
    
    errors={}
    n=0
    for i in list(hist):
        normalised_hist = i/(normed_value*widths)
        log_hist = np.log(normalised_hist)
        sqrt_hist = math.sqrt(i)
        if sqrt_hist == 1:
            upper_error = 1
            lower_error = 0.9
        if sqrt_hist !=1:
            upper_error = sqrt_hist
            lower_error = sqrt_hist
        normalised_upper_error = upper_error/(normed_value*widths)
        normalised_lower_error = lower_error/(normed_value*widths)
        errors[n]=(normalised_hist[0], normalised_upper_error[0], normalised_lower_error[0])
        n = n+1

    errors_corrected ={}
    for k, v in errors.items():
        binheight = v[0]
        log_binheight = np.log(v[0])
        upper_error = v[1]
        lower_error = v[2]
        log_upper_error = (np.log(upper_error+binheight))-log_binheight
        log_lower_error = log_binheight-(np.log(binheight-lower_error))
        errors_corrected[k] = (log_binheight, log_upper_error, log_lower_error)

    lower_err=[]
    upper_err=[]
    for k, v in errors_corrected.items():
        lower_error = v[2]
        upper_error = v[1]
        lower_err.append(lower_error)
        upper_err.append(upper_error)

    err = [tuple(lower_err),tuple(upper_err)]
    
    return err

In [6]:
# Upload muation rate from HSC from Lee-Six et al’s.
#Nucleotide content from whole-genome sequencing data(to a depth of 15X) on 140 single-cell 
#derived HSC colonies from a healthy 59-year-old male.

tri_nt="../../../Paper_data/Tri_nt_context_HSC/Trinucleotide_context_mutation_rates.csv"
tri_nt = pd.read_csv(tri_nt, sep=",")
tri_nt.columns = ['change_F', 'trint', 'change_R', '', 'sum_seq', 'num_seen', '', 'mutrate', '', '']
tri_nt = tri_nt[~tri_nt['change_F'].isna()]
tri_nt

,change_F,trint,change_R,,sum_seq,num_seen,,mutrate,,
0,A[C>A]A,2.0131,T[G>T]T,2.0205,4.0336,2510.0,NaN,1.332230e-09,NaN,NaN
1,A[C>A]C,1.1622,G[G>T]T,1.1636,2.3258,1030.0,NaN,9.481170e-10,NaN,NaN
2,A[C>A]G,0.2510,C[G>T]T,0.2516,0.5026,250.0,NaN,1.064910e-09,NaN,NaN
3,A[C>A]T,1.6076,A[G>T]T,1.6097,3.2173,1140.0,NaN,7.585960e-10,NaN,NaN
5,C[C>A]A,1.8426,T[G>T]G,1.8462,3.6888,2650.0,NaN,1.538010e-09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
113,G[T>G]T,1.4607,A[A>C]C,1.4548,2.9155,230.0,NaN,1.688930e-10,NaN,NaN
115,T[T>G]A,2.0814,T[A>C]A,2.0782,4.1596,302.0,NaN,1.554360e-10,NaN,NaN
116,T[T>G]C,1.9728,G[A>C]A,1.9703,3.9431,290.0,NaN,1.574550e-10,NaN,NaN
117,T[T>G]G,1.8981,C[A>C]A,1.8905,3.7886,502.0,NaN,2.836760e-10,NaN,NaN


# Select 

In [5]:
# Upload mutations (mutaitons in cases with only a single mutaitons) dataframe including 
#the flanking nt per mutation (NOT PROVIDED)
# File generated in 6_Fitness_analysis/1_variants_distribution.ipynb
file_name="boostDM_all_context_nonHNALL_unique.txt.gzip"

In [6]:
#used_individuals_500k = 151794
limit_500k = 10**(-1.44891098) # limit from DNAMT3A
used_individuals_500k = 467202
# DNMT3A R882H mutation rate
DNMT3A_R882H_mu = 1.88229e-08
binmethod = "doane"

In [7]:
#create a dictionary of DNMT3A R288H (with row_count as key) old set of mutaitons
with gzip.open(file_name,'rt',encoding='utf-8') as file:
    row_count = 0
    DNMT3A_R288H_all={} #dictionary of DNMTA missense and nonsense results
    for line in file:
        line = line.strip("\r\n").split("\t")
        if row_count>0: #ignore the header of the CSV file
            if line[12]=='DNMT3A': #only look at the DNMT3A results
                VAF = line[9] #VAF
                ID = line[-11] #participant ID
                age_group = line[-4]
                mutation_type = line[-6] #whether the variant is missense or nonsense or synonymous
                if mutation_type == '2-25234373-T':
                    DNMT3A_R288H_all[row_count]=(ID, VAF, mutation_type, age_group)
        row_count=row_count+1
len(DNMT3A_R288H_all)

877

In [8]:
#DNMT3A_R288H_all

In [9]:
DNMT3A_R288H_all_list = mut_dict2mut_list(DNMT3A_R288H_all)
DNMT3A_R288H_VAF_densities = normalised_VAF_normalised_densities_list(DNMT3A_R288H_all_list, used_individuals_500k, DNMT3A_R882H_mu, binmethod)
DNMT3A_R288H_VAF_densities

[-3.8918203  -3.65619483 -3.42056936 -3.18494389 -2.94931842 -2.71369295
 -2.47806748 -2.24244201 -2.00681654 -1.77119107 -1.5355656  -1.29994013
 -1.06431466 -0.82868919 -0.59306372]
[  3.  34. 119. 133.  97.  96.  72.  67.  65.  57.  51.  43.  26.  14.]


[(-3.77400756324964, 6.584651467333986, 0.8612115025164906, 0.455746394408326),
 (-3.5383820935276633,
  9.012399703282037,
  0.18813673438313216,
  0.15828377123345128),
 (-3.3027566238056876,
  10.265162671777407,
  0.09614736481902852,
  0.08770849617316401),
 (-3.067131154083712,
  10.37638830688763,
  0.09070290625528976,
  0.08315570061990663),
 (-2.831505684361735,
  10.06075015716926,
  0.1070671005050059,
  0.09670431346945918),
 (-2.5958802146397595,
  10.050387370133713,
  0.10765433624941778,
  0.09718303638212156),
 (-2.360254744917783,
  9.762705297681931,
  0.12539445059434762,
  0.11140820861960776),
 (-2.124629275195807,
  9.690731798056843,
  0.13030169298808936,
  0.1152638156235497),
 (-1.8890038054738312,
  9.660426448561513,
  0.1324288401894247,
  0.11692465365346116),
 (-1.653378335751855,
  9.529090446500426,
  0.1420858616474856,
  0.12438628454808409),
 (-1.4177528660298788,
  9.417864811390201,
  0.15085545817516177,
  0.13105283087898378),
 (-1.182127396307

In [10]:
DNMT3A_R288H_all_trimmed_list = []

for k, v in DNMT3A_R288H_all.items():
    if float(v[1])>=limit_500k and float(v[1])<= 0.5:
        DNMT3A_R288H_all_trimmed_list.append(v[1])
#DNMT3A_R288H_all_trimmed_list
len(DNMT3A_R288H_all_trimmed_list)

805

In [11]:
DNMT3A_R288H_VAF_densities = normalised_VAF_normalised_densities_list(DNMT3A_R288H_all_trimmed_list, used_individuals_500k, DNMT3A_R882H_mu, binmethod)

[-3.33220451 -3.14523657 -2.95826864 -2.7713007  -2.58433276 -2.39736482
 -2.21039689 -2.02342895 -1.83646101 -1.64949307 -1.46252514 -1.2755572
 -1.08858926 -0.90162132 -0.71465339]
[112. 106.  81.  78.  62.  55.  53.  57.  46.  41.  42.  31.  22.  19.]


In [12]:
# R882H densities (normalised by 2 x mu)
R882H_densities_without_inf = []    #remove the densities for the VAF bins for which there were no densities
for i in DNMT3A_R288H_VAF_densities:
    if i[1] != -(float('inf')):
        R882H_densities_without_inf.append(i)

In [13]:
#MLE function for range of ages
#l=ln(f)

def logProbtheory_ages(l, params): #= predicted density (i.e. normalised by 2 x mu)
    "Natural log of the probability of observing a variant within a specific binwidth if able to sequence perfectly"
    total_density=0.0
    N=params[0]
    s=params[1]
    sigma = params[2]
    mean=60
    dt=0.1

    total_density=integrate.quad(lambda t: (N/(1-2*np.exp(l))*np.exp(-((np.exp(l))/(((np.exp(s*t)-1)/(2*N*s))*(1-2*np.exp(l))))))*\
                                 ((1/((2*np.pi*(sigma)**2)**0.5))*(np.exp(-(t-mean)**2/(2*(sigma**2))))), mean-2*sigma, mean+2*sigma)
    return np.log(total_density[0])

def logProbDataGivenModel_ages(params, data):
    "This returns the natural log likelihood of the entire data, in specified binwidths, for a given theta and phi"
    total_square_distance = 0
    ages = np.linspace(38, 72, 35)
    for age in ages:
        for datapoint in data:
            if np.exp(datapoint[0])<0.5:
                logfreq = datapoint[0]
                predicted_log_density = logProbtheory_ages(logfreq, params)
                square_distance = ((datapoint[1] - predicted_log_density)**2)
                total_square_distance = total_square_distance + square_distance
    return total_square_distance   

In [14]:
ages = np.linspace(38, 72, 35) #age range for all the stuides together
initial_guess=[100000, 0.15, 10] #initial guess for [N\tau, s, sigma] (where sigma = standard deviation of ages)
scipy.optimize.minimize(logProbDataGivenModel_ages, initial_guess, args=(R882H_densities_without_inf,), method='Nelder-Mead')   

/tmp/ipykernel_181/3282986836.py:13: RuntimeWarning: overflow encountered in exp
  total_density=integrate.quad(lambda t: (N/(1-2*np.exp(l))*np.exp(-((np.exp(l))/(((np.exp(s*t)-1)/(2*N*s))*(1-2*np.exp(l))))))*\
/tmp/ipykernel_181/3282986836.py:13: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  total_density=integrate.quad(lambda t: (N/(1-2*np.exp(l))*np.exp(-((np.exp(l))/(((np.exp(s*t)-1)/(2*N*s))*(1-2*np.exp(l))))))*\


 final_simplex: (array([[8.97360377e+04, 9.25348780e-02, 2.99964385e+01],
       [8.97360377e+04, 9.25348780e-02, 2.99964385e+01],
       [8.97360377e+04, 9.25348780e-02, 2.99964384e+01],
       [8.97360377e+04, 9.25348780e-02, 2.99964385e+01]]), array([10.87152717, 10.87152717, 10.87152717, 10.87152717]))
           fun: 10.871527168566168
       message: 'Optimization terminated successfully.'
          nfev: 320
           nit: 174
        status: 0
       success: True
             x: array([8.97360377e+04, 9.25348780e-02, 2.99964385e+01])

# The outputs that we will need to infer fitness for all mutaitons

N 𝜏  = 8.97360377e+04
s (for DNMT3A R882H) = 9.25348780e-02
standard deviation of ages = 2.99964385e+01